<div class="alert alert-block alert-warning">
Only Python 3 compatible! Python 2 support has been dropped since v3.0 of this UseCase.
</div>

# Hippocampus Single Cell Validation Use Case

***
**Aim: ** To validate single cell hippocampal models against data from literature.

***
**Version:** 3.1 (13/01/2020)
***
**Contributors:**  Sáray Sára (KOKI), Shailesh Appukuttan (CNRS)
***
**Contact:** [saray.sara@koki.mta.hu](mailto:saray.sara@koki.mta.hu), [shailesh.appukuttan@unic.cnrs-gif.fr](mailto:shailesh.appukuttan@unic.cnrs-gif.fr)

<div class="alert alert-block alert-success" style="color:black">
<h2>About This Use Case</h2><br />
This use case shall take as input a BluePyOpt optimized output file. The validation tests run on the hoc template specified as "best\_cell" in the `meta.json` file. There are a total of six different tests in this use case:
<br/><br/>

<dl>
<dt>1. __Somatic Features Test - UCL data set__:  _(for pyramidal cells and interneurons)_</dt>
<dd>It evaluates the model against various eFEL features under somatic current injection of varying amplitudes. The experimental dataset used for validation is obtained from UCL. The This test can be used for both pyramidal cells and interneurons.<br/><br/></dd>

    
<dt>2. __Somatic Features Test - JMakara data set__:  _(for pyramidal cells)_</dt>
<dd>It evaluates the model against various eFEL features under somatic current injection of varying amplitudes. The experimental dataset used for validation is obtained from Judit Makara. The This test can be used only for pyramidal cells.<br/><br/></dd>

<dt>3. __Depolarization Block Test__:  _(for pyramidal cells)_</dt>
<dd>The Depolarization Block Test aims to determine whether the model enters depolarization block to prolonged, high intensity somatic current stimulus. It compares the current intensity to which the model fires the maximum number of action potentials, the current intensity before the model enters depolarization block (the two should be equal) and the equilibrium potential during depolarization block to the experimental data of Bianchi et al. 2012 (http://dx.doi.org/10.1007/s10827-012-0383-y).<br/><br/></dd>

<dt>4. __Back-Propagating AP Test__:  _(for pyramidal cells)_</dt>
<dd>The Back-Propagating AP Test evaluates the mode and efficacy of back-propagating action potentials on the apical trunk in locations of different distances from the soma. The amplitude of the first and last AP of around 15 Hz train is compared to experimental data from Golding et al. 2001 (https://doi.org/10.1152/jn.2001.86.6.2998).<br/><br/></dd>

<dt>5. __PSP Attenuation Test__:  _(for pyramidal cells)_</dt>
<dd>The PSP Attenuation Test evaluates how much the post synaptic potential (using EPSC stimulus) attenuates from the dendrite (different distances) to the soma. The soma/dendrite attenuation is compared to data from Magee & Cook 2000 (http://dx.doi.org/10.1038/78800).<br/><br/></dd>
    
<dt>6. __Oblique Integration Test__:  _(for pyramidal cells)_</dt>
<dd>Tests signal integration in oblique dendrites for increasing number of synchronous and asynchronous synaptic inputs. The experimental data is obtained from Losonczy and Magee 2006 (https://doi.org/10.1016/j.neuron.2006.03.016).<br/></dd>
</dl>

The results are registered on the HBP Validation Framework app. If an instance of the Model Catalog and Validation Framework are not found in the current Collab, then these are created. Additionally, a test report is generated and this can be viewed within the Jupyter notebook.
</div>

## 1. Seting up the environment

In [ ]:
#Loading the proper version of NEURON
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version
!pip install -q tornado==4.5.3
!pip install -q --upgrade hbp-service-client

<div class="alert alert-block alert-danger">
<strong>Note:</strong> If you encounter any errors in the below cell, please try to restart the kernel (Kernel -> Restart & Run All)
</div>

In [ ]:
req_packages = [    
                    {"sciunit"                  : {"min_version": "0.2.1",  "install_version": "0.2.1"}},
                    {"neuronunit"               : {"min_version": "0.1.8.2","install_version": "0.1.8.2"}},
                    {"hippounit"                : {"min_version": "1.3.3",  "install_version": "1.3.3"}},
                    {"bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"}},                    
                    {"hbp_validation_framework" : {"min_version": "0.5.28", "install_version": "0.5.28"}},
                    {"numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"}},    
                    {"Jinja2"                   : {"min_version": "2.10.3", "install_version": "2.10.3"}},                    
                    {"tornado"                  : {"min_version": "4.5.3",  "install_version": "4.5.3"}}
                ]

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg in req_packages:        
        for pkg_name, pkg_vinfo in pkg.items():
            print("Checking for package: {}".format(pkg_name))        
            try:
                pkg_resources.get_distribution(pkg_name)        
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                print("\t{}: current version = {}".format(pkg_name, current_version))
                if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):                                                
                        print("\tInstalling another version of {}.".format(pkg_name))
                        raise
            except:            
                if "github.com" in pkg_vinfo["install_version"]:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall git+{}".format(pkg_vinfo["install_version"]))
                else:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                print("\t{}: installed version = {}".format(pkg_name, pkg_vinfo["install_version"]))

install_req_packages()                 

In [ ]:
import sciunit
import bluepyopt.ephys as ephys
import hippounit
from hippounit.utils import ModelLoader_BPO
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

import json
import requests
import urllib
from io import StringIO, BytesIO
import zipfile
import collections
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings('ignore')

## 2. Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME)
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

## 3. Model Selection: Specifying model from ModelCatalog
Hippocampus models registered on the Model Catalog and known to be in the BluePyOpt format are listed below.

In [ ]:
l1 = modelCatalog.list_models(brain_region="hippocampus", collab_id="12027")
l2 = modelCatalog.list_models(brain_region="hippocampus", collab_id="9821")
list_of_models = []
list_of_models.extend(l1)
list_of_models.extend(l2)

for item in list_of_models:
    if item["author"][0]["family_name"] not in ["Migliore", "Vitale"]:
        list_of_models.remove(item)
print(len(list_of_models))      
df = pd.DataFrame.from_dict(json_normalize(list_of_models), orient='columns')
df = df.reindex(columns=['name', 'id', 'author', 'brain_region', 'species', 'cell_type', 'model_scope', 'abstraction_level', 'description'])
df.index += 1
df.replace('\n','', regex=True)

In [ ]:
print("\nEnter the # of required model: ")
choice = int(input())
if choice <= len(list_of_models) and choice > 0:
    model_id = list_of_models[choice-1]["id"]
    model_name = list_of_models[choice-1]["name"]
    model_instances = modelCatalog.list_model_instances(model_id=model_id)    
    model_instance_json = max(model_instances, key=lambda x:x['timestamp'])
    file_path = model_instance_json["source"]
else:
    raise ValueError("Invalid entry for model choice!")

try:        
    response = requests.get(file_path)
    zip_ref = zipfile.ZipFile(BytesIO(response.content))
    zip_ref.extractall()
    model_path = os.path.join(os.getcwd(),model_name)

    meta_info = requests.get(file_path.replace(".zip", "_meta.json"))    
    if meta_info.status_code == 200:
        with open(os.path.join(model_path, model_name+"_meta.json"), 'w') as f:
            json.dump(meta_info.json(), f)
    
    model_image_url = file_path.replace(model_name+".zip", "_".join(model_name.split("_")[3:-1])+"_morph.jpeg")
    model_image = requests.get(model_image_url)    
    model_image_localPath = None
    if model_image.status_code == 200:        
        model_image_localPath = os.path.join(model_path, model_name.split("_")[3]+"_morph.jpg")
        with open(model_image_localPath, 'wb') as f:
            f.write(model_image.content)         
        print("\nModel Morphology:")
        img = mpimg.imread(model_image_localPath)
        imgplot = plt.imshow(img)
        plt.show()
except:
    raise IOError("Model url = {} is invalid!".format(file_path))

### 3.1 Gather Additional Info
Some of the tests are applicable only for pyramidal cells, while others are more generic (also applicable for interneurons). Hence it is essential to identify the cell type: _pyramidal_ or _interneuron_ <br />
The usecase tries to identify the cell type from the model name. If this isn't possible, the user is prompted to specify the cell type.

In [ ]:
# will try to determine cell type from file name; if not then ask user
# current options: msn_d1, msn_d2
if "_pyr_" in model_name:
    cell_type = "pyramidal cell" 
elif "_int_" in model_name:
    cell_type = "interneuron"
else:
    print("\nPlease enter the cell_type: ")
    options = ["pyramidal cell", "interneuron"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = input()
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))

## 4. Selecting the Validation Test

In [ ]:
test_list = {"Somatic Features Test - UCL data set"     : ["pyramidal cell", "interneuron"],
             "Somatic Features Test - JMakara data set" : ["pyramidal cell"],
             "Back-Propagating AP Test"                 : ["pyramidal cell"], 
             "PSP Attenuation Test"                     : ["pyramidal cell"], 
             "Depolarization Block Test"                : ["pyramidal cell"],
             "Oblique Integration Test"                 : ["pyramidal cell"]}
valid_test_list = [key for key,val in test_list.items() if cell_type in val]
    
print("\nPlease choose from the following available tests for your cell type: ")
for i, each in enumerate(valid_test_list,start=1):
        print("\t{}. {}".format(i,each))
print("Note: you may specify multiple tests: ")
print("Example inputs: 1, 4, [1,4], all")
try:           
    test_entry = raw_input().lower()
except (AttributeError, NameError):
    test_entry = input().lower()
try:
    if test_entry == "all":
        select_test_list = valid_test_list
    else:    
        if isinstance(eval(test_entry), list):
            test_entry = eval(test_entry)
            test_entry = [x - 1 for x in test_entry] # menu numbering was from 1            
            select_test_list = [valid_test_list[i] for i in test_entry]
        elif isinstance(eval(test_entry), int):
            test_entry = eval(test_entry) - 1 # menu numbering was from 1
            select_test_list = [valid_test_list[test_entry]]
        else:
            raise ValueError("Invalid entry for test selection!")
except ValueError:
    print("Invalid entry for test selection!")

## 5. Instantiating the model; Running model validation for selected tests
The usecase will run validations on the specified model for each of the specified tests. At the end of the test, the user is provided with a textual summary of the score and the path to related output files generated by the tests. These and other details can be viewed in the Validation Framework app (see Collab's Navigation panel; select Validation Framework).

In [ ]:
cell_model = ModelLoader_BPO(name=model_name, model_dir=model_path, SomaSecList_name = "somatic")
cell_model.model_uuid = model_id
cell_model.source = file_path
cell_model.model_version = "1.0"

result_uuids = []

for test in select_test_list:
    if test == "Somatic Features Test - UCL data set":
        specify_data_set = 'UCL_data'

        if 'CA1_pyr_cACpyr' in model_name:                          
            ttype = "CA1_pyr_cACpyr"        
        elif 'CA1_int_bAC' in model_name:
            ttype = "CA1_int_bAC"
        elif 'CA1_int_cAC' in model_name:
            ttype = "CA1_int_cAC"
        elif 'CA1_int_cNAC' in model_name:
            ttype = "CA1_int_cNAC"
            
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/somafeat_stim/stim_" + ttype + ".json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_somafeat_"+ttype, test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config, specify_data_set=specify_data_set)
            
    elif test == "Somatic Features Test - JMakara data set":
        specify_data_set = 'JMakara_data'
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/somafeat_stim/stim_rat_CA1_PC_JMakara.json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_somafeat_CA1_pyr_patch", test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config, specify_data_set=specify_data_set)            
    
    elif test == "Back-Propagating AP Test":
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/bAP_stim/stim_bAP_test.json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_ca1_bap", test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config)            
        
    elif test == "PSP Attenuation Test":
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/PSP_attenuation_stim/stim_PSP_attenuation_test.json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_ca1_psp_attenuation", test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config)
        
    elif test == "Depolarization Block Test":        
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_ca1_depolblock", test_version="1.0", storage_collab_id=collab_id, register_result=True)            
        
    elif test == "Oblique Integration Test":
        result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="hippo_ca1_obl_int", test_version="1.0", storage_collab_id=collab_id, register_result=True)
               
    result_uuids.append(result_id)
    
print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### 5.1 Score Summary

In [ ]:
if len(result_uuids) > 0:
    df, excluded_results = utils.generate_score_matrix(result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)        
    from IPython.core.display import HTML
    HTML("<style>.rendered_html th {max-width: 120px;}</style>")
    display(df)

### 5.2 Generate Report
The validation framework can generate an HTML report for all the successfully completed tests. The user is prompted whether such a report should be generated for the current validation results. If asked to generate, the location to the generated HTML is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate an HTML report of the executed tests?")
    print("Enter: y/n")
    choice = input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        report_path, valid_uuids = utils.generate_HTML_report(result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)    

### 5.3 View Report Inside Jupyter Notebook
The HTML report created in the above cell is displayed within the Jupyter notebook.

In [ ]:
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width="100%", height=1000))